In [1]:
import os, sys

import numpy as np
import pandas as pd

import anndata
import scanpy as sc
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt

In [2]:
path = '/Users/busracagirici/Documents/scrnaseq/citeseq/'
save_tables = '/Users/busracagirici/Documents/scrnaseq/citeseq/tables/'
save_figures = '/Users/busracagirici/Documents/scrnaseq/citeseq/figures/'

adata = sc.read_h5ad(path + 'data/merged_dengue_and_covid_all.h5ad')

filtered = adata[(adata.obs.Dataset.isin(['Children_viscRNAseq', 'Adults_citeseq'])) & 
                 (adata.obs.cell_quality == 'high')]

filtered = filtered[(filtered.obs.ID.isin([#'1-056-01', '1-172-01', '1_003_1', '1_053_01', '1_183_01',
                                          #'1_001_1', '1_013_01', '1_026_02', '1_036_02', '5_049_01',
                                           '1_019_01', '6_023_01', '5_030_01', '5_193_01', '5_154_01', 
                                            '6_001_01', '5_041_01', '1_140_01', '1_144_01', '5_044_01', 
                                            '1_002_01', '1_113_01', '5_010_01', '5_130_01'
                                          ]))].copy()
filtered

AnnData object with n_obs × n_vars = 69471 × 42244
    obs: 'Age', 'Condition', 'DENV_minus', 'DENV_plus', 'DENV_reads', 'Gender', 'ID', 'batch', 'cell_quality', 'cell_subtype', 'cell_subtype_2', 'cell_subtype_zhiyuan', 'cell_type', 'cell_type_zhiyuan', 'cell_type_new_2', 'doublets', 'mt_frac', 'n_counts', 'n_genes', 'platform', 'viral_load_nano', 'viral_load_qpcr', 'Dengue_exposure', 'cell_subtype_luca', 'cell_subtype_new', 'cell_type_luca', 'cell_type_new', 'cell_subtype_NK', 'Dataset', 'Admission', 'Sample', 'Stage', 'Experiment', 'Date', 'Date of symptoms onset', 'Days', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'predicted_doublets', 'mad_prd', 'n_proteins_by_counts', 'total_counts_adt', 'total_counts_log1p', 'total_counts_adt_log1p', 'rna_quality', 'protein_quality', 'high_quality', 'TCR_umi', 'TCR_umi_log1p', 'TCR_IR_VJ_2_c_call', 'TCR_IR_VDJ_2_c_call', 'TCR_IR_VJ_2_d_call', 'TCR_IR_VDJ_2_d_call', 'TCR_IR_VJ_2_v_call', 'TCR_IR_VDJ_2

In [3]:
sc.pp.normalize_total(filtered, target_sum=1e6)
sc.pp.log1p(filtered, base=2)
filtered.raw = filtered
sc.pp.filter_genes(filtered, min_cells=3)


In [6]:
def med_pair(filtered, obs='Condition', obs_keys = ['S_dengue', 'dengue'], ct_obs = 'rna_ct_2_2', ct='NK cells', ncells=5):
    adata_ct = filtered[filtered.obs[ct_obs] == ct].copy()
    IDs=[]
    for ID in adata_ct.obs.ID.unique():
        if adata_ct[adata_ct.obs.ID == ID].obs.shape[0] >= ncells: IDs.append(ID)
    adata_ct = adata_ct[adata_ct.obs['ID'].isin(IDs)]
        
    adata_S_ct = adata_ct[adata_ct.obs[obs].isin([obs_keys[0]])]
    adata_D_ct = adata_ct[adata_ct.obs[obs].isin([obs_keys[1]])]
    
    df_S = sc.get.obs_df(adata_S_ct, keys=[*adata_ct.var_names, 'ID']).groupby(['ID']).mean().dropna().T
    df_D = sc.get.obs_df(adata_D_ct, keys=[*adata_ct.var_names, 'ID']).groupby(['ID']).mean().dropna().T
    
    df_new = pd.DataFrame()
    for col in df_S.columns:
        df1 = df_D.T - df_S[col].T
        df1.index = [col+'__'+x for x in df1.index]
        df_new = pd.concat([df_new, df1], axis=0)
    df_new = df_new * -1
    df_new = df_new
    if ct == 'conventional DCs': df_new['cell_type'] = 'cDCs'
    else: df_new['cell_type'] = ct
    
    df_new['SD'] = [x.split('__')[0] for x in df_new.index]
    df_new['D'] = [x.split('__')[1] for x in df_new.index]
    df_new = df_new.reset_index().set_index(['cell_type', 'SD', 'D'])
    del(df_new['index'])
    return(df_new)


In [11]:
filtered.obs.cell_type_new.cat.categories

Index(['B cells', 'Doublets', 'Monocytes', 'NK cells', 'Plasmablasts',
       'T cells', 'cDCs', 'megakaryocytes', 'pDCs', 'unknown', 'unknown2'],
      dtype='object')

In [17]:
cts = ['B cells', 'Monocytes', 'NK cells', 'Plasmablasts', 'T cells', 'cDCs', 'pDCs',]
df_all = pd.DataFrame([])
for ct in cts:
    df_all = pd.concat([df_all, med_pair(filtered, obs='Condition', obs_keys = ['SD', 'D'], 
                                         ct_obs = 'cell_type_new', ct=ct, ncells=5)], axis=0)
df_all

A1BG  A1BG-AS1       A2M   A2M-AS1    A4GALT  \
cell_type SD       D                                                            
B cells   1_002_01 1_019_01  0.088832  0.020684  0.007482 -0.026178  0.017331   
                   1_113_01 -0.388926  0.070605  0.007482 -0.012049  0.017331   
                   5_010_01 -0.201013 -0.040747  0.007482 -0.000000  0.002952   
                   5_130_01 -0.220403 -0.036726  0.007482 -0.006422 -0.003078   
                   5_154_01  0.119057  0.117594  0.007482 -0.005724  0.017331   
...                               ...       ...       ...       ...       ...   
pDCs      5_193_01 5_010_01  1.454234 -0.211923 -0.000000 -0.000000 -0.000000   
                   5_130_01  2.683112 -0.900882 -0.000000 -0.000000 -0.000000   
                   5_154_01  3.550779 -0.384325 -0.000000 -0.000000 -0.000000   
                   6_001_01  2.801996 -0.173012 -0.000000 -0.000000 -0.000000   
                   6_023_01  2.059881 -0.291907 -0.000000 -0.000000 -0.000000   

                                 AAAS      AACS  AADAT     AAED1     AAGAB  \
cell_type SD       D                                                         
B cells   1_002_01 1_019_01 -0.134464  0.021115   -0.0  0.102176  0.003819   
                   1_113_01 -0.169468  0.060608   -0.0  0.078169  0.168751   
                   5_010_01 -0.220292 -0.010794   -0.0  0.091779  0.042159   
                   5_130_01 -0.159050  0.041595   -0.0  0.017678  0.073891   
                   5_154_01 -0.021795  0.065730   -0.0  0.176432  0.091145   
...                               ...       ...    ...       ...       ...   
pDCs      5_193_01 5_010_01 -0.767089 -0.601583   -0.0 -0.222762  0.974675   
                   5_130_01 -1.502071 -0.000000   -0.0 -1.493941  2.828889   
                   5_154_01 -0.584723 -0.196620   -0.0 -0.178491  2.619710   
                   6_001_01 -0.843893 -0.000000   -0.0 -0.000000  2.056952   
                   6_023_01 -0.886426 -0.298460   -0.0 -0.151391  1.779694   

                             ...      ZW10    ZWILCH     ZWINT      ZXDA  \
cell_type SD       D         ...                                           
B cells   1_002_01 1_019_01  ...  0.021904 -0.038976  0.031843  0.043836   
                   1_113_01  ...  0.027573 -0.067950 -0.001988  0.022342   
                   5_010_01  ...  0.030020  0.020399 -0.006693  0.038291   
                   5_130_01  ...  0.009148  0.006036 -0.001571 -0.019530   
                   5_154_01  ...  0.009689  0.026202  0.026773  0.026327   
...                          ...       ...       ...       ...       ...   
pDCs      5_193_01 5_010_01  ... -0.000000 -0.000000 -0.148300 -0.000000   
                   5_130_01  ... -0.000000 -0.000000 -0.000000 -0.000000   
                   5_154_01  ... -0.393535 -0.000000 -0.000000 -0.000000   
                   6_001_01  ... -0.150400 -0.157617 -0.000000 -0.000000   
                   6_023_01  ... -0.000000 -0.000000 -0.000000 -0.000000   

                                 ZXDB      ZXDC    ZYG11A    ZYG11B       ZYX  \
cell_type SD       D                                                            
B cells   1_002_01 1_019_01 -0.016432  0.021503 -0.000000  0.063782  0.295725   
                   1_113_01 -0.023051  0.042908 -0.000000  0.098926  0.044852   
                   5_010_01 -0.013502  0.067379 -0.027775  0.135168 -0.160236   
                   5_130_01 -0.048885  0.038933 -0.012371  0.125272 -0.048513   
                   5_154_01 -0.006908  0.020091 -0.018181  0.137464  0.186208   
...                               ...       ...       ...       ...       ...   
pDCs      5_193_01 5_010_01 -0.193205  1.530440 -0.000000 -0.378447  4.516590   
                   5_130_01 -0.694177  1.530440 -0.000000 -0.000000  2.696577   
                   5_154_01 -0.000000  1.340830 -0.000000 -0.000000  5.819514   
                   6_001_01 -0.000000  1.530440 -0.000000 -0.000000  5.847435   
   

In [19]:
#df_all.to_csv(save_tables+'DEGs_SD_vs_D_medpair_raw_adata_kid.csv', sep='\t')

In [18]:
degs = df_all.groupby(['cell_type']).median()
degs

,A1BG,A1BG-AS1,A2M,A2M-AS1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
cell_type,,,,,,,,,,,,,,,,,,,,,
B cells,0.069665,-0.010789,-0.000000,-0.007035,-0.0,-0.020605,0.041595,-0.0,0.094645,0.050895,...,0.021091,0.026202,-0.000000,0.029743,0.023561,0.042908,-0.000000,0.071278,0.186208,0.020681
Monocytes,0.254801,0.099574,-0.000000,0.004852,-0.0,0.004309,0.077023,-0.0,0.097469,-0.053163,...,0.042775,-0.042202,-0.000000,-0.000000,0.024790,-0.000000,-0.000000,0.008232,-0.067420,0.014514
NK cells,0.136194,0.003759,-0.000000,0.003739,-0.0,0.032561,0.113198,-0.0,0.189697,0.201730,...,0.132891,0.058611,0.126113,-0.018145,-0.006984,0.012493,-0.000000,0.053103,0.180898,0.045093
Plasmablasts,0.327529,0.052680,-0.035977,-0.000000,-0.0,0.149684,0.201231,-0.0,0.131412,0.117899,...,0.156217,0.140020,0.602750,0.004199,0.009737,-0.000543,0.004498,0.054082,-0.026187,0.089176
T cells,0.082524,-0.005433,-0.000000,-0.006364,-0.0,0.009278,0.061005,-0.0,0.055143,0.238673,...,0.126509,0.098219,0.162887,0.025225,-0.008953,-0.006799,-0.000000,0.114546,-0.402758,0.142024
cDCs,0.044180,0.008430,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,-0.0,-0.156295,0.354412,...,0.042585,-0.000000,0.246812,-0.000000,-0.000000,0.392357,-0.000000,-0.129435,0.066077,0.548220
pDCs,0.741799,-0.211923,-0.000000,-0.000000,-0.0,-0.105168,0.122934,-0.0,-0.151391,0.061888,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.920327,-0.198108


In [19]:
degs['SPON2']

cell_type
B cells         0.005863
Monocytes      -0.063447
NK cells       -1.298629
Plasmablasts   -0.053194
T cells        -0.167395
cDCs           -0.000000
pDCs            0.191053
Name: SPON2, dtype: float32

In [22]:
filtered = adata[(adata.obs.Dataset.isin(['Children_viscRNAseq', 'Adults_citeseq'])) & 
                 (adata.obs.cell_quality == 'high')]

filtered = filtered[(filtered.obs.ID.isin(['1-056-01', '1-172-01', '1_003_1', '1_053_01', '1_183_01',
                                          '1_001_1', '1_013_01', '1_026_02', '1_036_02', '5_049_01',
                                          # '1_019_01', '6_023_01', '5_030_01', '5_193_01', '5_154_01', 
                                           # '6_001_01', '5_041_01', '1_140_01', '1_144_01', '5_044_01', 
                                           # '1_002_01', '1_113_01', '5_010_01', '5_130_01'
                                          ]))].copy()

sc.pp.normalize_total(filtered, target_sum=1e6)
sc.pp.log1p(filtered, base=2)
filtered.raw = filtered
sc.pp.filter_genes(filtered, min_cells=3)


In [23]:
cts = ['B cells', 'Monocytes', 'NK cells', 'Plasmablasts', 'T cells', 'cDCs', 'pDCs',]
df_all = pd.DataFrame([])
for ct in cts:
    df_all = pd.concat([df_all, med_pair(filtered, obs='Condition', obs_keys = ['SD', 'D'], 
                                         ct_obs = 'cell_type_new', ct=ct, ncells=5)], axis=0)

#df_all.to_csv(save_tables+'DEGs_SD_vs_D_medpair_raw.csv', sep='\t')
df_all

A1BG  A1BG-AS1  A1CF       A2M   A2M-AS1  \
cell_type SD       D                                                        
B cells   1_001_1  1-056-01 -0.620257 -0.089538  -0.0 -0.026183 -0.000000   
                   1-172-01 -0.260756 -0.118432  -0.0 -0.000000 -0.000000   
                   1_003_1  -0.217618  0.058586  -0.0 -0.039802 -0.017236   
                   1_053_01 -0.394277 -0.072280  -0.0 -0.000000 -0.000000   
                   1_183_01 -0.237109 -0.096643  -0.0 -0.000000 -0.000000   
...                               ...       ...   ...       ...       ...   
pDCs      1_026_02 1_183_01  1.657682  0.434463  -0.0 -0.199965 -0.000000   
          1_036_02 1-172-01 -0.357307 -0.383732  -0.0  0.198931 -0.339524   
                   1_003_1   1.189888  0.513668  -0.0  0.198931 -0.100202   
                   1_053_01 -0.005646  0.306670  -0.0  0.105275 -0.097924   
                   1_183_01  1.420046  0.513668  -0.0 -0.001034 -0.000000   

                             A2ML1  A2ML1-AS1   A3GALT2    A4GALT      AAAS  \
cell_type SD       D                                                          
B cells   1_001_1  1-056-01   -0.0  -0.091882 -0.000000 -0.006653 -0.680724   
                   1-172-01   -0.0   0.035315 -0.000000 -0.000000 -0.445314   
                   1_003_1    -0.0   0.027492 -0.007607 -0.000000 -0.002409   
                   1_053_01   -0.0  -0.107122 -0.000000 -0.000000 -0.177466   
                   1_183_01   -0.0  -0.000302 -0.000000 -0.000000 -0.347500   
...                            ...        ...       ...       ...       ...   
pDCs      1_026_02 1_183_01   -0.0  -0.208594 -0.000000 -0.000000 -0.208005   
          1_036_02 1-172-01   -0.0  -0.000000 -0.000000 -0.085087 -0.137428   
                   1_003_1    -0.0  -0.000000 -0.000000 -0.000000  0.870737   
                   1_053_01   -0.0  -0.000000 -0.000000 -0.998982  0.894258   
                   1_183_01   -0.0  -0.208594 -0.000000 -0.000000  1.273094   

                             ...    ZWILCH     ZWINT      ZXDA      ZXDB  \
cell_type SD       D         ...                                           
B cells   1_001_1  1-056-01  ... -0.021303  0.006038 -0.130119 -0.190559   
                   1-172-01  ... -0.077583 -0.005506 -0.032458 -0.338899   
                   1_003_1   ...  0.015362  0.037056  0.030149 -0.082769   
                   1_053_01  ...  0.075208  0.045503 -0.159010 -0.169278   
                   1_183_01  ... -0.046422  0.024751  0.014739 -0.047658   
...                          ...       ...       ...       ...       ...   
pDCs      1_026_02 1_183_01  ...  0.211177 -0.202757 -0.202881 -0.199568   
          1_036_02 1-172-01  ... -0.054743 -0.188312  0.229443 -0.082838   
                   1_003_1   ... -0.199034  0.099988  0.217111 -0.201030   
                   1_053_01  ... -0.282371  0.004196  0.315342 -0.095164   
                   1_183_01  ... -0.113050 -0.102769  0.112461 -0.199568   

                                 ZXDC    ZYG11A    ZYG11B       ZYX     ZZEF1  \
cell_type SD       D                                                            
B cells   1_001_1  1-056-01 -0.295899 -0.039681  0.017176 -0.217162  0.115468   
                   1-172-01  0.090169 -0.039251  0.107566 -0.682973  0.593638   
                   1_003_1   0.176517 -0.057938  0.228013 -0.414240  0.527319   
                   1_053_01 -0.101062 -0.000949  0.189678 -0.296900  0.683933   
                   1_183_01  0.288498 -0.003128  0.265339 -0.127198  0.662157   
...                               ...       ...       ...       ...       ...   
pDCs      1_026_02 1_183_01  2.774392 -0.000000 -0.646791 -0.943135 -0.196432   
          1_036_02 1-172-01 -0.333400 -0.000000 -0.002589 -0.363374 -0.556997   
                   1_003_1   0.592155 -0.000000 -0.319548  1.582684 -0.320618   
                   1_053_01 -0.279448 -0.000000 -0.195714 -0.083774  0.149828   
                   1_183_01  1.286477 -0.0

In [24]:
degs = df_all.groupby(['cell_type']).median()
degs

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,hsa-mir-423
cell_type,,,,,,,,,,,,,,,,,,,,,
B cells,0.071736,0.000644,-0.000000,-0.000000,-0.000000,-0.0,0.020662,-0.000000,-0.000000,-0.132458,...,0.013109,-0.000000,-0.033581,-0.055697,-0.135797,0.004438,0.046761,0.152036,0.077111,-0.0
Monocytes,-0.079274,-0.093507,-0.000000,0.015628,0.000032,-0.0,0.002626,0.022048,-0.000000,-0.127406,...,0.064698,0.005157,-0.003808,-0.030278,-0.077224,-0.000000,0.014514,0.448051,0.292130,-0.0
NK cells,-0.018529,-0.002597,-0.000000,0.134581,-0.034244,-0.0,-0.005872,-0.000000,-0.000000,-0.149544,...,0.020466,-0.032579,-0.028307,0.014832,-0.017648,-0.000000,0.088003,0.031675,-0.194883,-0.0
Plasmablasts,-0.286548,0.060843,0.003222,-0.021994,0.025228,-0.0,0.005584,-0.000000,-0.000000,-0.569538,...,-0.071341,0.133855,0.018116,0.014049,-0.280789,-0.036671,0.092350,-0.396991,0.174218,-0.0
T cells,-0.064061,-0.029399,-0.000000,0.024328,-0.072244,-0.0,-0.010080,0.003158,-0.000000,-0.018528,...,0.119224,0.297776,0.003040,0.027570,-0.018367,0.001485,0.006098,0.382944,0.241144,-0.0
cDCs,0.679325,0.043491,-0.000000,-0.011428,0.053480,-0.0,0.031022,0.066133,-0.000000,-0.905610,...,-0.042902,-0.000826,0.117323,-0.021838,0.125557,-0.000000,-0.096872,0.294509,-0.442967,-0.0
pDCs,-0.062659,0.267068,-0.000000,-0.000000,-0.097924,-0.0,-0.000000,-0.000000,-0.042543,-0.208005,...,0.186261,-0.099280,0.028105,0.004087,0.589407,-0.000000,-0.294915,-0.446866,0.134655,-0.0


In [25]:
degs['SPON2']

cell_type
B cells        -0.081215
Monocytes      -0.050700
NK cells       -0.922317
Plasmablasts    0.000293
T cells         0.110600
cDCs           -0.296151
pDCs           -0.250181
Name: SPON2, dtype: float32